<a href="https://colab.research.google.com/github/DOMINION-JOHN1/voting_eligibility_model/blob/main/preprocessing_of_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
# Define the file path of the np.array document
file_path = '/content/drive/My Drive/.npy'

# Load the np.array document using np.load()
loaded_array = np.load(file_path)

In [4]:
import tensorflow as tf

x=loaded_array
x.astype('float16')

array([[[0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        ...,
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ]],

       [[0.851 , 0.8667, 0.8667, ..., 0.757 , 0.745 , 0.749 ],
        [0.859 , 0.863 , 0.851 , ..., 0.749 , 0.753 , 0.7646],
        [0.8237, 0.8354, 0.8315, ..., 0.749 , 0.745 , 0.7646],
        ...,
        [0.1647, 0.1647, 0.153 , ..., 0.569 , 0.5845, 0.569 ],
        [0.1686, 0.1765, 0.149 , ..., 0.5767, 0.565 , 0.565 ],
        [0.1569, 0.1647, 0.153 , ..., 0.5605, 0.5566, 0.5728]],

       [[0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        ...,
        [0. 

In [5]:

file_path = '/content/drive/My Drive/'

# Save the numpy array to a .npy file with the name 'feature_data.npy'
file_name = 'feature_data.npy'
np.save(file_path + file_name, x)

print(f"Numpy array saved to '{file_path}{file_name}'")


Numpy array saved to '/content/drive/My Drive/feature_data.npy'


In [6]:
eligibility=pd.read_csv('/content/drive/My Drive/eligibility.csv')

In [9]:

eligibility['eligibility']=eligibility['eligibility'].replace('55', 'Eligible')
eligibility['eligibility']=eligibility['eligibility'].replace('35', 'Eligible')
eligibility['eligibility'] = eligibility['eligibility'].replace({'Eligible': 1, 'Not Eligible': 0})
target_label = np.array(eligibility['eligibility'])
target_label.astype('float16')

array([1., 1., 1., ..., 1., 1., 1.], dtype=float16)

In [10]:

# Save the numpy array to a .npy file with the name 'feature_data.npy'
file_name = 'target_data.npy'
np.save(file_path + file_name, target_label)

print(f"Numpy array saved to '{file_path}{file_name}'")

Numpy array saved to '/content/drive/My Drive/target_data.npy'
